# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [ ]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
             
{
  "tableName": "projects",
  "fields": [
    {
      "name": "ID_project",
      "type": "int"
    },
    {
      "name": "project_name",
      "type": "varchar"
    },
    {
      "name": "start_date",
      "type": "date"
    }
  ]
}

             
{
  "tableName": "salaries",
  "fields": [
    {
      "name": "ID_salary",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "amount",
      "type": "decimal"
    },
    {
      "name": "pay_date",
      "type": "date"
    }
  ]
}


{
  "tableName": "attendance",
  "fields": [
    {
      "name": "ID_attendance",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "check_in",
      "type": "datetime"
    },
    {
      "name": "check_out",
      "type": "datetime"
    }
  ]
}
            
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
what is the name of the project?
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [7]:
#new
context_user = context.copy()
print(return_CCRMSQL("""SELECT * FROM projects""", context_user))

Sure, here is the SQL query to retrieve all the data from the "projects" table:

```sql
SELECT * FROM projects;
```


In [8]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("SELECT name FROM employees", old_context_user))

This is your SQL:
```sql
SELECT name FROM employees;
```

Explanation: This SQL query selects the "name" column from the "employees" table, retrieving the names of all employees in the database.


In [9]:
#new
print(return_CCRMSQL(" Select s.pay_date, a.check_in, a.check_out From salaries s Join attendance a ON s.ID_usr = a.ID_usr ", context_user))

SELECT s.pay_date, a.check_in, a.check_out 
FROM salaries s 
JOIN attendance a ON s.ID_usr = a.ID_usr


In [10]:
#old
print(return_CCRMSQL(" Select e.name, s.salary From employees e Join salary s ON e.ID_usr = s.ID_usr ", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr;
```

This SQL query selects the name from the employees table and the salary from the salary table, joining them on the ID_usr column to match the data from both tables.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative


In [16]:
print(return_CCRMSQL("SELECT s.ID_usr, s.amount, p.project_name, p.start_date FROM salaries s JOIN projects p ON s.ID_usr = p.ID_project", old_context_user))


This is your SQL:
```sql
SELECT s.ID_usr, s.amount, p.project_name, p.start_date 
FROM salaries s 
JOIN projects p ON s.ID_usr = p.ID_project;
```

This SQL query selects the user ID, salary amount, project name, and project start date from the "salaries" table (aliased as "s") and the "projects" table (aliased as "p"), joining them on the condition that the user ID matches the project ID.


In [15]:
print(return_CCRMSQL("SELECT a.ID_usr, a.check_in, a.check_out, s.amount, s.pay_date FROM attendance a JOIN salaries s ON a.ID_usr = s.ID_usr ORDER BY s.pay_date DESC", old_context_user))


This is your SQL:
```sql
SELECT a.ID_usr, a.check_in, a.check_out, s.amount, s.pay_date 
FROM attendance a 
JOIN salaries s ON a.ID_usr = s.ID_usr 
ORDER BY s.pay_date DESC;
```

This SQL query selects the user ID, check-in time, check-out time, salary amount, and payment date from the "attendance" and "salaries" tables, respectively. It then joins the two tables on the user ID and orders the results by the payment date in descending order.


In [ ]:
print(return_CCRMSQL("SELECT s.ID_usr, s.amount, p.project_name, p.start_date FROM salaries s JOIN projects p ON s.ID_usr = p.ID_project", old_context_user))


This is your SQL:
```sql
SELECT e.name, s.salary 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr;
```

Explanation: This SQL query selects the name of employees from the "employees" table and their corresponding salary from the "salary" table, joining the two tables on the "ID_usr" column.


 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [ ]:
"""After testing 3 versions of AI, I find it smart enught to recognize queries.

I leard that AI is smart enught to know whether the conversation is fictinal or not, that was very impressive!  """